#### Использованные библиотеки

In [ ]:
!pip install -r requirements.txt

In [ ]:
# это нужно для одной из моделей, но не факт, что успешно поставится
!pip install learn2learn

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import pytorch_lightning as pl
from torch import nn
from torch import optim
from torch.nn import functional as F
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from torchvision.datasets import MNIST, FashionMNIST, CIFAR10, KMNIST, SVHN
from models import MLP, LSTM, Transformer
from datagen import SubLoader, TaskAugmentor
from torch.utils.data import DataLoader

import logging
from IPython.display import clear_output
logging.getLogger("pytorch_lightning.utilities.rank_zero").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.accelerators.cuda").setLevel(logging.WARNING)

import warnings
warnings.filterwarnings("ignore")

torch.set_float32_matmul_precision('medium')
device = "cuda" if torch.cuda.is_available() else "cpu"
device

#### Setup

Чтобы понять, насколько трансформер вообще хорош, нужно понять, насколько остальные модели плохи. Идея метатеста в статье довольно странная из того, что я понял. Суть в том, что модель должна увидеть 99 примеров из одной таски и сделать предсказание на 100. Если всё хорошо - модель научилась извлекать новую информацию. Если нет - то нет. При этом алгоритм немного разный, в зависимости от модели, но обо всём по порядку. Чёткого описания в статье разумеется нет. Референсом служит вот такая таблица

<img width="700px" src="https://cdn.discordapp.com/attachments/674191702906503199/1194436697258196992/image.png?ex=65b058dc&is=659de3dc&hm=730df59553279d76ef794f920bbe03b5f2837cf8424e1ccb77711d36d5676bcf&">

Я буду делать для всех датасетов, кроме рандомного, потому что я так и не понял, как его нужно собирать. Там было что-то про сэмплирование из равномерного распределения, но я уже как-то слишком устал

Для метатеста нужно "просмотреть" 99 примеров и сделать предсказание на 100. Для rnn это более-менее понятно - сделать предсказание с маской, а вот для перцептрона не так очевидно. Как я понимаю, достаточно сделать GD на 99 и предсказать на 100, так и поступим

In [7]:
from utils import (
    g_transform, cifar_transform, svhn_transform,
    mlp_params, lstm_params, gpt_params
)
from models import MLP, GPT, LSTM
# from learn2learn.algorithms import LightningMAML

#### Train

Функция трейна будет очень большая, потому что алгоритм отличается сильно. Что нам нужно сделать, так это удостоваериться, что датасет прочитается, какой бы он ни был, за это отвечает `get_test_sets`, написать аккураси для rnn - предсказывать по 99 100-й, и аккураси для mlp - выучить 99, предсказать 100-й. Возвращают они немного разное, как-то лень это всё унифицировать, сделаем позже

In [ ]:
def get_test_sets(dataset, transform):
    try:
        to_see = SubLoader(dataset('./datasets', split="train", transform=transform)) 
        to_predict = SubLoader(dataset('./datasets', split="test", transform=transform)) 
    except:
        to_see = SubLoader(dataset('./datasets', train=True, transform=transform)) 
        to_predict = SubLoader(dataset('./datasets', train=False, transform=transform))
    return to_see, to_predict

def sequence_last_acc_rnn(model, loader, model_name):
    if model_name == "gpt":
        attention_mask = torch.zeros(len(loader), 100)
        attention_mask[:, :99] = 1
        pred = F.softmax(
            model(loader.dataset.data, attention_mask=attention_mask), 1
        ).argmax(1)
    elif model_name == "lstm":
        pred = F.softmax(model(loader.dataset.data), 1).argmax(1)
    res = (pred[:, -1] == loader.dataset.targets[:, -1]).float().mean()
    return {"valid_accuracy": res.item()}
    
def sequence_last_acc_mlp(to_see, to_predict):
    trainer = pl.Trainer(
        enable_model_summary=False,
        enable_progress_bar=False,
        max_epochs=1, accelerator="gpu",
    )
    trainer.fit(model, test_to_see, test_to_predict)
    return trainer.callback_metrics

И наконец функция трейна. Ведёт себя по-разному в зависимости от модели. Можно фиксировать сид, я делал на 3 разных, варьировать батчи, число тестовых тасок здесь это замена сиду - они все разные, и суффикс для сохранения результатов

In [10]:
def last_accuracy_evaluation(
    model_name,
    random_state=69,
    batch_size=128,
    n_train_tasks=2**16,
    n_test_tasks=256,
    output_file_suffix="",
    **trainer_arguments
):   
    logs = []
    # draw train distribution which is always augmented mnist
    mnist = SubLoader(MNIST('./datasets', train=True, transform=g_transform))
    draw_sequence = True if model_name in ["lstm", "gpt"] else False
    train_augmentor = TaskAugmentor(
        n_tasks=n_train_tasks,
        random_state=random_state,
        draw_sequence=draw_sequence
    )
    train = train_augmentor.transform(mnist, n_samples=1)
    meta_train = DataLoader(train, batch_size=batch_size, shuffle=True)

    # define model among given choices or add your own
    model = {
        "mlp": MLP(**mlp_params),
        # "maml": LightningMAML(MLP(**mlp_params), lr=0.1), requires learn2liearn lib
        "lstm": LSTM(**lstm_params),
        "gpt": GPT(**gpt_params)
    }[model_name]

    # training config differs for each model
    training_config = {"max_steps": 100000}
    if model_name in ["mlp", "maml"]:
        training_config = {"max_epochs": 10}
    trainer = pl.Trainer(
        enable_model_summary=False,
        accelerator="gpu",
        callbacks=[EarlyStopping(
            monitor="train_accuracy",
            min_delta=0.025,
            patience=100,
            mode="max"
        )], **training_config, **trainer_arguments
    )
    trainer.fit(model, meta_train)

    # calculate accuracy on unseen datasets
    for dataset, transform, dataset_name in zip(
        [MNIST, FashionMNIST, KMNIST, CIFAR10, SVHN],
        [g_transform]*3+[cifar_transform, svhn_transform],
        ["MNIST", "FashionMNIST", "KMNIST", "CIFAR10", "SVHN"]
    ):
        test_augmentor = TaskAugmentor(
            n_tasks=n_test_tasks,
            draw_sequence=draw_sequence,
            random_state=random_state-1
        )
        # to see is either an unseen sequence or sequence to memorize for mlp
        to_see, to_predict = get_test_sets(dataset, transform)
        to_see_samples = 99 if model_name in ["mlp", "maml"] else 1
        to_see = test_augmentor.transform(to_see, n_samples=to_see_samples)
        to_see = DataLoader(to_see, batch_size=1, shuffle=False)

        # perform GD for mlp or browse for rnn
        if model_name in ["lstm", "gpt"]:
            last_accuracy_entry = sequence_last_acc_rnn(model, to_see, model_name)
        elif model_name in ["mlp", "maml"]:
            to_predict = test_augmentor.transform(to_predict, n_samples=1)
            to_predict = DataLoader(to_predict, batch_size=1, shuffle=False)
            last_accuracy_entry = sequence_last_acc_mlp(to_see, to_predict)

        # store and update logs
        entry = {
            "model_name": model_name,
            "trained_on": train_dataset_name,
            "dataset": dataset_name,
        }
        entry.update(last_accuracy_entry)
        logs.append(entry)
        pd.DataFrame(logs).to_csv(
            f"experiments/metatest_{model_name}{output_file_suffix}.csv", index=0
        )
        
    clear_output(True)
    return logs

Я делал для 16 тасок в случае перцептрона - вспоминаем график из `2. Hparams sweep.ipynb`, там этого было достаточно, чтобы обучиться, а для rnn беру $2^{14}$, там как раз происходит фазовый переход

In [ ]:
for model in ["mlp", "maml", "lstm", "gpt"]:
    n_tasks = 16 if model in ["mlp", "maml"] else 2**14
    last_accuracy_evaluation(model_name, n_train_tasks=n_tasks)

#### Interpretaion
Теперь приятная часть

In [55]:
import pandas as pd

df = pd.concat([
    pd.read_csv(f"experiments/metatest_{dataset}.csv")
    for dataset in ["mlp", "lstm", "gpt"]
])

In [90]:
col_order = [
    ('valid_accuracy', x) for x in
    ["MNIST", "FashionMNIST", "KMNIST", "CIFAR10", "SVHN"]
]
index_order = ["mlp", "lstm", "gpt"]

summary = (
    df.groupby(["model_name", "dataset"]) \
    .mean("valid_accuracy") \
    .reset_index() \
    .pivot(index="model_name", columns="dataset") \
    # .reindex(col_order, axis=1) \
    # .reindex(index_order, axis=0)
)
summary.loc[index_order, col_order]

valid_accuracy                                           
dataset             MNIST FashionMNIST    KMNIST   CIFAR10      SVHN
model_name                                                          
mlp              0.370968     0.225806  0.096774  0.081967  0.049180
lstm             0.109375     0.095052  0.098958  0.104167  0.102865
gpt              0.523438     0.458333  0.350260  0.114583  0.088542

Мои результаты вышли не такими впечатляющими с точки зрения цифр, но одну вещь в них всё равно видно. MLP ещё что-то может из себя выдавить, если датасет похож, потому что он его тупо запоминает. LSTM ни рыба, ни мясо, хотя он был у меня не такой, как в статье. А вот трансформер это совсем другое дело, хотя на цветных датасетах он всё-таки не очень себя показал